# **Example Road Network**

Roads are vital to the infrastructure of a community and, when blocked and/or (partially) destroyed 
cause numerous issues for various players. Therefore, it is imperative to assess the impact of loss in mobility when analyzing flood events and other hazards. 

FIAT offers a tool to compute the monetary impact of the road blockage/destruction based on the **extent of the hazard, the road type (e.g. motorway, trunk, primary roads), and the number of lanes.**

Users can either download data from Open Street Map (OSM) through the tool or ingest their own road network data.

OSM offers detailed information about the road network such as the roadtype and the number of lanes, which are obligatory to calculate the flood impact. **This notebook will cover the road impact assessment based on OSM.**

>*Note: If the user wants to use their own data they musst assure to deliver the **number of lanes per segment***.

## Let's get started!

# **Step 1**: Import required packages

In [5]:
# First, all required packages must be installed.

from hydromt_fiat.fiat import FiatModel
from hydromt.log import setuplog
from pathlib import Path
import pytest
import shutil
import geopandas as gpd
import pandas as pd
import os 
import json
import yaml

# **Step 2:** Define the study area

FIAT requires the input of a region to download the OSM road dataset. This can be done via vectorfile of a polygon or GeoJSON object. We recommened to download the region from [Geojson.io](https://geojson.io/#map=2/0/20). 

Draw a polygon on the map and create a polygon from the GeoJson Object. When you download the file path, refer to it in the "region" tab in the *yaml* file. You can also copy the GeoJson code directly into the configuration file (see example in [configuration file](C:/Users/rautenba/hydromt_fiat/examples/data/road_network/configuration.yml))

# **Step 3:** Create a configuration file 

FIAT requires a configuration file in form of a *yaml*-file or a dictionary, in which the model parameters are defined. The configuration file must define the vulnerability and the exposure setup.
The required fields are:
  
*setup_road_vulnerability:*
- **vertical_unit**:Unit of vertical inundation (feet or meter)
- **threshold_value**: Threshold inundation height at which road is fully destroyed 
- **min_hazard_value**: minimum inundation height (recommended: 0)
- **max_hazard_value**: maximum inundation height (e.g. 6 meters) 
- **step_hazard_value**: discrete levels or steps of inundation height

*setup_exposure_roads:*
- **roads_fn**: File path to road network file or OSM data (for OSM data use: "OSM")
- **road_types**: Road type keys from OSM (e.g. "motorway", "primary")
- **road_damage**: "default_road_max_potential_damages",
- **unit**: Unit of road length (feet or meter)

*region:*
- **region**: pre-defined region in form of a dictionary or shapely polygon

The specified parameters must be ingested into the *yaml* construct. Please check [here](C:/Users/rautenba/hydromt_fiat/examples/data/road_network/configuration.yml) for the setup of the *yaml* file. 

## **Step 3.1:** Load the information of the <em>yaml</em> file

The *yaml* file is located in the "example"/ "data" / "road_network"  - folder.  
The information in the file are converted into a python dictionary. </p>
<p><em>Note: Instead of loading a yaml file it is possible to create a python dictionary directly, built upon the structure seen below.</em></p>

In [124]:
#Let's read the yaml file with the required information 
#and set up the FIAT model for the road network.

with open(Path(os.path.abspath("")) / "data" / "road_network" / "configuration.yml", 'r') as file:
    _cases = yaml.safe_load(file)

print(json.dumps(_cases, indent=4, sort_keys=False))

{
    "cases": {
        "roads_from_OSM": {
            "data_catalogue": "C:/Users/rautenba/hydromt_fiat/examples/data/road_network/hydromt_fiat_catalog_USA.yml",
            "dir": "test_roads_from_OSM"
        },
        "road_network": {
            "new_root": "./data/road_network/output/road_network_example",
            "configuration": {
                "setup_road_vulnerability": {
                    "vertical_unit": "feet",
                    "threshold_value": 0.6,
                    "min_hazard_value": 0,
                    "max_hazard_value": 10,
                    "step_hazard_value": 1
                },
                "setup_exposure_roads": {
                    "roads_fn": "OSM",
                    "road_types": [
                        "motorway",
                        "primary",
                        "secondary",
                        "tertiary"
                    ],
                    "road_damage": "default_road_max_potential_damages",
           

# **Step 4**: <strong>Define variables for the FIAT model</strong>

Set up the root path to the FIAT model and the logger settings to re-build your model:
- `root`: Directory path from where the neccessary data for the FIAT model is stored.  </p>
- `logger`: The logger variable defines the frequencies of log-entries during the initialisation of the model.
- `region`: Define the region as a variable from the case dictionary.
- `data_catalog_yml`: Define the data catalogue as a variable from the case dictionary.</p>



In [118]:
# Set up Fiat Model
root = Path(os.path.abspath(""))  / "data" / "road_network"

# Set up the logger
logger = setuplog("hydromt_fiat", log_level=10)

# Set up region
region = _cases["cases"]["region"]

# Set up datacatalog
data_catalog_yml = str(_cases["cases"]["roads_from_OSM"]["data_catalogue"])

2023-10-23 16:03:50,225 - hydromt_fiat - log - INFO - HydroMT version: 0.8.0


INFO:hydromt_fiat:HydroMT version: 0.8.0


# **Step 4:** Run the FIAT model

It's time to run the model with the new configuration of the riad network. The output will be 
stored in the root-directory. The output will be an **exposure folder** incl. *exposure.csv* together with vector data files, a **hazard folder** (empty) and a **vulnerability folder** incl. *damage_curve.csv*. 

In [119]:
fm = FiatModel(root=root, mode="w+", data_libs=[data_catalog_yml], logger=logger)
region = gpd.GeoDataFrame.from_features(_cases["cases"]["region"], crs=4326)
fm.build(region={"geom": region}, opt=_cases["cases"]["road_network"]["configuration"])
fm.write()

2023-10-23 16:03:52,501 - hydromt_fiat - data_catalog - INFO - Parsing data catalog from ./hydromt_fiat_catalog_USA.yml


INFO:hydromt_fiat:Parsing data catalog from ./hydromt_fiat_catalog_USA.yml


FileNotFoundError: [Errno 2] No such file or directory: './hydromt_fiat_catalog_USA.yml'

# **Done!**
Your **FIAT model** created the exposure and vulnerability files for the road network. Using that data in combination with a hazard map, the impact for the defined hazard can be computed. 

Let's have a look at the output!